# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [17]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/k3/x2ny3m2d3y57h8xp7cr2qxv80000gn/T/pip-req-build-zkgrwiqk
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/k3/x2ny3m2d3y57h8xp7cr2qxv80000gn/T/pip-req-build-zkgrwiqk
^C
ERROR: Operation cancelled by user


In [24]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

ImportError: cannot import name 'MarketBreadthFeatureEngineer' from 'finrl.meta.preprocessor.preprocessors' (/Users/pu17/miniconda3/envs/finrobot/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py)

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [ ]:
# aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [ ]:
# aapl_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.876099,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.167603,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.742645,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.400543,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.565208,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [20]:
sz_df_finrl = YahooDownloader(start_date = '2023-07-01',
                                end_date = '2024-10-07',
                                ticker_list = [
                                    # 'BABA', 'PDD',  
                                    # '^HSI',
                                                '399001.SZ',
                                                # '002594.SZ','300750.SZ','000333.SZ','000858.SZ',
   #                                                 '002594.SZ',  # 比亚迪
   #  '300750.SZ',  # 宁德时代
   #  '000333.SZ',  # 美的集团
   #  '000858.SZ',  # 五粮液
   #  '600036.SZ',  # 招商银行
   #  '002475.SZ',  # 立讯精密
   #  '002352.SZ',  # 顺丰控股
   #  '603259.SH',  # 药明康德
   #  '000001.SZ'   # 平安银行
                                                #     'MCHI',       # MSCI中国指数
                                                #     'FXI',        # iShares中国大盘ETF
                                                #     'KWEB',        # KraneShares中国互联网ETF
                                                #  '^GSPC',      # 标普500指数
                                                #     '^NDX',       # 纳斯达克100指数
                                                #     '^DJI',       # 道琼斯工业平均指数
                                                #     '^IXIC',      # 纳斯达克综合指数
                                                #     '^RUT',       # 罗素2000指数
                                                    ]  
).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (305, 8)


In [21]:
sz_df_finrl.tail(20)

,date,open,high,low,close,volume,tic,day
285,2024-08-30,8152.810059,8453.660156,8152.810059,8348.480469,1719900,399001.SZ,4
286,2024-09-02,8331.459961,8340.379883,8172.209961,8172.209961,1347400,399001.SZ,0
287,2024-09-03,8157.439941,8294.769531,8157.439941,8268.049805,1133200,399001.SZ,1
288,2024-09-04,8207.280273,8297.820312,8189.529785,8226.240234,1045600,399001.SZ,2
289,2024-09-05,8224.509766,8297.940430,8220.549805,8249.660156,954400,399001.SZ,3
290,2024-09-06,8252.009766,8260.690430,8128.770020,8130.770020,964900,399001.SZ,4
291,2024-09-09,8086.279785,8129.540039,8023.370117,8063.270020,991400,399001.SZ,0
292,2024-09-10,8064.399902,8113.689941,7973.790039,8073.830078,1023800,399001.SZ,1
293,2024-09-11,8052.250000,8139.700195,8031.950195,8105.379883,946100,399001.SZ,2
294,2024-09-12,8122.770020,8176.049805,8053.879883,8054.240234,980100,399001.SZ,3


## Data for the chosen tickers

In [ ]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [ ]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (94301, 8)


In [ ]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.737005,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,41.816158,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.053309,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,30.712521,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [ ]:
import tushare as ts
import pandas as pd
import numpy as np

# 设置 Tushare API token
ts.set_token('4bccdd4d130c436773beef521fbecc05ab0079026122c370908f3c93')
pro = ts.pro_api()

# 获取某一天的所有股票的日线数据（你可以更改 trade_date）
# 例如获取 2023-01-01 的所有股票的日线数据
market_data = pro.daily(trade_date='20240930')

# 查看数据结构
print(market_data.head())

# 计算上涨和下跌家数
market_data['up'] = (market_data['pct_chg'] > 0).astype(int)  # pct_chg 是Tushare返回的每日涨跌幅列
market_data['down'] = (market_data['pct_chg'] < 0).astype(int)

# 按日期统计上涨股票数和下跌股票数
daily_up = market_data.groupby('trade_date')['up'].sum()
daily_down = market_data.groupby('trade_date')['down'].sum()

# 将统计结果转换为 DataFrame
market_breadth = pd.DataFrame({
    'trade_date': daily_up.index,
    'up_count': daily_up.values,
    'down_count': daily_down.values
})

# 计算上涨/下跌比率
market_breadth['up_down_ratio'] = market_breadth['up_count'] / (market_breadth['down_count'] + 1e-6)  # 避免除以0

# 计算市场宽度指标（上涨股票数 - 下跌股票数）
market_breadth['market_breadth'] = market_breadth['up_count'] - market_breadth['down_count']

# 查看市场宽度和广度信息
print(market_breadth)

     ts_code trade_date   open   high    low  close  pre_close  change  \
0  000001.SZ   20240930  11.62  12.30  11.56  12.21      11.42    0.79   
1  000002.SZ   20240930   9.72   9.72   9.27   9.72       8.84    0.88   
2  000004.SZ   20240930  14.94  16.30  14.94  16.30      14.82    1.48   
3  000006.SZ   20240930   5.49   5.60   5.21   5.59       5.13    0.46   
4  000007.SZ   20240930   6.42   6.77   6.25   6.77       6.15    0.62   

   pct_chg         vol       amount  
0   6.9177  5430283.64  6467438.838  
1   9.9548  5797265.70  5592761.873  
2   9.9865   405823.65   636048.463  
3   8.9669  1256420.88   684436.800  
4  10.0813   194520.91   128200.050  
  trade_date  up_count  down_count  up_down_ratio  market_breadth
0   20240930      5336           8     666.999917            5328


In [22]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(sz_df_finrl)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (313, 8)
Successfully added vix
Successfully added turbulence index


In [23]:
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2023-07-03,11059.309570,11128.929688,11028.240234,11091.559570,1321600,399001.SZ,0,0.000000,11185.628458,10981.810996,27.048893,-100.000000,81.246307,11091.559570,11091.559570,13.57,0.0
1,2023-07-05,11129.120117,11129.120117,11014.389648,11029.299805,1054800,399001.SZ,2,-2.024506,11185.628458,10981.810996,27.048893,-100.000000,81.246307,11083.719727,11083.719727,14.18,0.0
2,2023-07-06,11001.870117,11072.070312,10954.150391,10968.370117,1067100,399001.SZ,3,-5.490836,11197.111454,10912.653194,18.586940,-126.022632,92.275534,11054.882324,11054.882324,15.44,0.0
3,2023-07-07,10939.290039,10949.980469,10852.959961,10888.549805,1032300,399001.SZ,4,-10.435025,11214.760972,10828.470669,13.052958,-135.458589,96.243043,11021.615820,11021.615820,14.83,0.0
4,2023-07-10,10926.360352,10995.799805,10901.839844,10942.830078,958400,399001.SZ,0,-10.889954,11192.827394,10824.142333,28.110302,-67.231368,57.168922,11008.484863,11008.484863,15.07,0.0


In [ ]:
from

In [19]:
import tushare as ts
import pandas as pd

# 设置 Tushare API token（请替换为你自己的）
ts.set_token('4bccdd4d130c436773beef521fbecc05ab0079026122c370908f3c93')
pro = ts.pro_api()

# 获取港股的基本信息
hk_basic = pro.hk_basic()

# 从获取的港股基本信息中提取 ts_code 列，获取所有港股的股票代码
ts_codes = hk_basic['ts_code'].tolist()

# 定义时间范围
start_date = '20230920'
end_date = '20230930'

# 创建一个空的 DataFrame 来存储所有股票的每日行情数据
all_data = pd.DataFrame()

# 遍历所有港股的 ts_code，获取它们的每日交易数据
for ts_code in ts_codes:
    try:
        # 获取每个股票的每日交易数据
        df = pro.hk_daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
        # 如果数据不为空，将其添加到 all_data 中
        if not df.empty:
            all_data = pd.concat([all_data, df])
    except Exception as e:
        print(f"Error fetching data for {ts_code}: {e}")

# 查看合并后的数据结构
print(all_data.head())

# 计算涨跌幅
all_data['pct_chg'] = all_data['close'].pct_change() * 100

# 将交易日期转换为日期时间格式
all_data['trade_date'] = pd.to_datetime(all_data['trade_date'])

# 计算上涨和下跌的家数
all_data['up'] = (all_data['pct_chg'] > 0).astype(int)
all_data['down'] = (all_data['pct_chg'] < 0).astype(int)

# 按交易日期统计每一天上涨和下跌的股票数量
daily_up = all_data.groupby('trade_date')['up'].sum()
daily_down = all_data.groupby('trade_date')['down'].sum()

# 将统计结果转换为 DataFrame
market_breadth = pd.DataFrame({
    'trade_date': daily_up.index,
    'up_count': daily_up.values,
    'down_count': daily_down.values
})

# 计算上涨/下跌比率
market_breadth['up_down_ratio'] = market_breadth['up_count'] / (market_breadth['down_count'] + 1e-6)  # 避免除以0

# 计算市场广度指标（上涨股票数 - 下跌股票数）
market_breadth['market_breadth'] = market_breadth['up_count'] - market_breadth['down_count']

# 查看市场广度和上涨/下跌比率信息
print(market_breadth)

KeyboardInterrupt: 

In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.737005,746015200.0,4.0,0.0,2.958118,2.631402,100.0,66.666667,100.0,2.737005,2.737005,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,41.816158,6547900.0,4.0,0.0,2.958118,2.631402,100.0,66.666667,100.0,41.816158,41.816158,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.053309,10955700.0,4.0,0.0,2.958118,2.631402,100.0,66.666667,100.0,15.053309,15.053309,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.0,2.958118,2.631402,100.0,66.666667,100.0,33.941101,33.941101,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,30.712521,7117200.0,4.0,0.0,2.958118,2.631402,100.0,66.666667,100.0,30.712521,30.712521,39.189999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [ ]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [ ]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')